In [1]:
import tweepy
import nltk
import pandas as pd
import numpy as np
import sys
import jsonpickle
import os
import keras
import tensorflow
from textblob import TextBlob 
from plotly.offline import init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
import csv
import gensim
import logging
import tempfile
from nltk.corpus import stopwords
from textblob import Word
from string import punctuation
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict
import pyLDAvis.gensim
from bs4 import BeautifulSoup
import re
import emoji
import time

init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities

import warnings
warnings.filterwarnings("ignore")
plotly.offline.init_notebook_mode()

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
# Consumer keys and access tokens, used for OAuth
consumer_key = '1K7KS692MTDzqWfHvvAPPSlqE'
consumer_secret = 'bRy95ycRGxry1EsSaQfnOVuZl31A5Sw6sn5xdXr2qcZkUmm0Aw'
access_token = '1053856790834753536-vLIZETTNJ9eFxsCtCNV4UCk0eOpj7g'
access_token_secret = 'gbJVvuKGYVJUAzXpWA2NhbaFezK5PBqJNfMhaObbXPDX7'

#auth = tweepy.OAuthHandler('pr0AH7Ot5sZmig4u3bA6j51ty', 'tNteF0tRlEjKJfkkWQaIv5myqT9oBqrIVOYPQJOMjBTJhn9SAF')
#auth.set_access_token('934846563825930241-yO5rosUB4x8eFMO0J7IXV1UZM0RzbgL', 'CbqfvlRonXo2JiIyxqCqeZynwkslNcDPmGFQ9KBEh8Mch')
 
#OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
# Creation of the actual interface, using authentication
api = tweepy.API(auth)

user = api.me()
 
print('Name: ' + user.name)
print('Location: ' + user.location)
print('Friends: ' + str(user.friends_count))

auth = tweepy.OAuthHandler('pr0AH7Ot5sZmig4u3bA6j51ty', 'tNteF0tRlEjKJfkkWQaIv5myqT9oBqrIVOYPQJOMjBTJhn9SAF')
auth.set_access_token('934846563825930241-yO5rosUB4x8eFMO0J7IXV1UZM0RzbgL', 'CbqfvlRonXo2JiIyxqCqeZynwkslNcDPmGFQ9KBEh8Mch')
api = tweepy.API(auth)

mashape_key = "uIX3UUkrh7mshux9VLXhN1FcUYY0p1ZEJpCjsnCHKddXFfIzhf"
twitter_app_auth = {
    'consumer_key': 'pr0AH7Ot5sZmig4u3bA6j51ty',
    'consumer_secret': 'tNteF0tRlEjKJfkkWQaIv5myqT9oBqrIVOYPQJOMjBTJhn9SAF',
    'access_token': '934846563825930241-yO5rosUB4x8eFMO0J7IXV1UZM0RzbgL',
    'access_token_secret': 'CbqfvlRonXo2JiIyxqCqeZynwkslNcDPmGFQ9KBEh8Mch',
  }

In [ ]:
# load legitimate users file
legit_df = pd.DataFrame.from_csv('legitimate_users.txt',header=None,sep='\t')
legit_df.reset_index(inplace=True)
legit_list = list(legit_df[0])

# load bot users 
bot_df = pd.DataFrame.from_csv('bot_users.txt',header=None,sep='\t')
bot_df.reset_index(inplace=True)
bot_list = list(bot_df[0])

# temporarily only use first 1000 bots and legit users
legit_list = legit_list[0:1000]
bot_list = bot_list[0:1000]

In [ ]:
def get_all_tweets(screen_name, bots_list_bool):
    
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
#     auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#     auth.set_access_token(access_token, access_token_secret)
#     api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    try:
        
        try:
            # if user id is inputted 
            val = int(screen_name)

            #make initial request for most recent tweets (200 is the maximum allowed count)
            new_tweets = api.user_timeline(user_id = screen_name,count=200)#, tweet_mode = 'extended')

        except ValueError:
            # input was a string (screen name)
            #make initial request for most recent tweets (200 is the maximum allowed count)
            new_tweets = api.user_timeline(screen_name = screen_name,count=200)#,tweet_mode = 'extended')

        #save most recent tweets
        alltweets.extend(new_tweets)

        #save the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        #keep grabbing tweets until there are no tweets left to grab
        while len(new_tweets) > 0:
            print("getting tweets before {}".format(oldest))

            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

            #save most recent tweets
            alltweets.extend(new_tweets)

            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1

            print("...{} tweets downloaded so far".format(len(alltweets)))

        screen_name = alltweets[0].user.screen_name
        user_id = alltweets[0].user.id_str
        
    
        #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[screen_name, user_id, tweet.id_str, tweet.text,tweet.source] for tweet in alltweets
                    ]

        # potential other features
        #tweet.created_at, tweet.retweets, tweet.retweet_count, tweet.retweeted,tweet.lang,  tweet.geo,
        #              tweet.favorite, tweet.favorite_count, tweet.favorited,tweet.place,tweet.entities,
        
        #"created_at","retweets","retweet_count",
        #                     "retweeted","favorite", "favorite_count", "favorited", "entities","lang","place","geo",

        #write the csv

        # choose file path
        if bots_list_bool:
            filepath ='data_NLP/bots/{}_tweets.csv'
        else:
            filepath ='data_NLP/legit/{}_tweets.csv'

        with open(filepath.format(user_id), 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["author","id","text","source"])
            writer.writerows(outtweets)

        pass
    except Exception as e: 
        print('Exception was:', e)
        print("Failed to load data for user {}".format(screen_name))
        pass
    

In [151]:
# get first 10 bot users
for i in range(19,30):
    print("--- Fetching Legit User Number {}---".format(i))
    get_all_tweets(bot_list[i], True)
    #time.sleep(30)

--- Fetching Legit User Number 19---
getting tweets before 236127121695711231
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 2157321
--- Fetching Legit User Number 20---
getting tweets before 457959017667510271
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 2219081
--- Fetching Legit User Number 21---
getting tweets before 1048221407325253632
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 2243161
--- Fetching Legit User Number 22---
getting tweets before 1044143902645788671
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 2269491
--- Fetching Legit User Number 23---
getting tweets before 895774831987314688
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 2695911
--- Fetching Legit User Number 24---
g

In [89]:
# get first 10 legit users
for i in range(500,700):
    print("--- Fetching Legit User Number {}---".format(i))
    get_all_tweets(legit_list[i], False)
    #time.sleep(30)
    


--- Fetching Legit User Number 500---
getting tweets before 1061389470698659845
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 7705112
--- Fetching Legit User Number 501---
getting tweets before 1065360401163083775
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 7737912
--- Fetching Legit User Number 502---
getting tweets before 756232235892613119
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 7760272
--- Fetching Legit User Number 503---
getting tweets before 950420201828360191
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 7760472
--- Fetching Legit User Number 504---
getting tweets before 624406885484118015
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 7777272
--- Fetching Legit User Number 50

getting tweets before 813229198508752896
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8208492
--- Fetching Legit User Number 546---
getting tweets before 1059281633046478847
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8240772
--- Fetching Legit User Number 547---
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8248602
--- Fetching Legit User Number 548---
getting tweets before 850499743402872831
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8254252
--- Fetching Legit User Number 549---
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8257322
--- Fetching Legit User Number 550---
getting tweets before 436708824503042047
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}

...201 tweets downloaded so far
--- Fetching Legit User Number 592---
getting tweets before 1057698790453506047
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8703862
--- Fetching Legit User Number 593---
getting tweets before 1061633827867480064
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8705472
--- Fetching Legit User Number 594---
getting tweets before 709397774819729407
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8719572
--- Fetching Legit User Number 595---
getting tweets before 238155669809790975
...201 tweets downloaded so far
getting tweets before 143350687462465536
...201 tweets downloaded so far
--- Fetching Legit User Number 596---
getting tweets before 1056685596570214399
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 8732352
--- Fetch

getting tweets before 1065384626737139711
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9279222
--- Fetching Legit User Number 638---
getting tweets before 959546225941991424
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9280282
--- Fetching Legit User Number 639---
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9287612
--- Fetching Legit User Number 640---
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9292972
--- Fetching Legit User Number 641---
getting tweets before 1033553261867937791
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9298152
--- Fetching Legit User Number 642---
getting tweets before 1022813283869503487
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.

getting tweets before 1059834086326460415
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9782772
--- Fetching Legit User Number 683---
getting tweets before 10804275298
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9800782
--- Fetching Legit User Number 684---
getting tweets before 1052554423342260223
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9804112
--- Fetching Legit User Number 685---
Exception was: Not authorized.
Failed to load data for user 9804712
--- Fetching Legit User Number 686---
getting tweets before 9576612756
Exception was: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to load data for user 9815272
--- Fetching Legit User Number 687---
Exception was: Not authorized.
Failed to load data for user 9820202
--- Fetching Legit User Number 688---
getting tweets before 1029170

In [101]:
from os import listdir
from os.path import isfile, join
!ls
mypath_bots = 'data_NLP/bots/'
mypath_legit = 'data_NLP/legit/'
botfiles = [f for f in listdir(mypath_bots) if isfile(join(mypath_bots, f)) and not f=='.DS_Store']
legitfiles = [f for f in listdir(mypath_bots) if isfile(join(mypath_bots, f))and not f=='.DS_Store']
print(legitfiles)

AM207_HW9_2018_Matthew-Stewart.ipynb bot_users.txt
BarackObama_tweets.csv               cresci-2017.csv (1)
LICENSE                              data_NLP
NLP_EDA.ipynb                        elonmusk_tweets.csv
README.md                            legitimate_users.txt
_config.yml                          tweets.json
['10997_tweets.csv', '1599001_tweets.csv', '4567451_tweets.csv', '7967132_tweets.csv', '6301_tweets.csv', '11228722_tweets.csv', '10836_tweets.csv']


In [114]:
get_all_tweets('elonmusk', False)

getting tweets before 1057695407344541696
...400 tweets downloaded so far
getting tweets before 1049363997617582080
...600 tweets downloaded so far
getting tweets before 1038332059129761792
...800 tweets downloaded so far
getting tweets before 1023749689701593087
...1000 tweets downloaded so far
getting tweets before 1015854883990265855
...1200 tweets downloaded so far
getting tweets before 1010428377797308415
...1400 tweets downloaded so far
getting tweets before 1005875421999554559
...1600 tweets downloaded so far
getting tweets before 1000444616536031231
...1800 tweets downloaded so far
getting tweets before 996102919811350527
...2000 tweets downloaded so far
getting tweets before 979895766880841732
...2200 tweets downloaded so far
getting tweets before 960390661211021311
...2400 tweets downloaded so far
getting tweets before 933576358793318400
...2600 tweets downloaded so far
getting tweets before 901904930071609344
...2800 tweets downloaded so far
getting tweets before 88562633450

In [118]:
!ls

tweets_df = pd.read_csv('elonmusk_tweets.csv')
# change to datetime
#tweets_df['created_at'] = pd.to_datetime(tweets_df['created_at'], format='%Y-%m-%d %H:%M:%S')
full_tweets =tweets_df.copy()
#tweetsT = tweets_df['created_at']
tweets_df = tweets_df[['text']]

AM207_HW9_2018_Matthew-Stewart.ipynb bot_users.txt
BarackObama_tweets.csv               cresci-2017.csv (1)
LICENSE                              data_NLP
NLP_EDA.ipynb                        elonmusk_tweets.csv
README.md                            legitimate_users.txt
_config.yml                          tweets.json


In [119]:
# get features of tweets (pre-cleaning data)

# number of hashtags 
tweets_df['num_hashtags'] = tweets_df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

# number of all-caps words 
tweets_df['all_upper'] = tweets_df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [144]:
tweets_df['text'].values[4]

'@margrethmpossi Varies per person, but about 80 sustained, peaking above 100 at times. Pain level increases exponentially above 80.'

In [132]:
print(len(tweets_df['text']))
tweets_df['text'].values[3223]

3224


"@verge It won't matter"

In [135]:
# deal with emojis

class Emoticons:
    POSITIVE = ["*O", "*-*", "*O*", "*o*", "* *",
                ":P", ":D", ":d", ":p",
                ";P", ";D", ";d", ";p",
                ":-)", ";-)", ":=)", ";=)",
                ":<)", ":>)", ";>)", ";=)",
                "=}", ":)", "(:;)",
                "(;", ":}", "{:", ";}",
                "{;:]",
                "[;", ":')", ";')", ":-3",
                "{;", ":]",
                ";-3", ":-x", ";-x", ":-X",
                ";-X", ":-}", ";-=}", ":-]",
                ";-]", ":-.)",
                "^_^", "^-^"]

    NEGATIVE = [":(", ";(", ":'(",
                "=(", "={", "):", ");",
                ")':", ")';", ")=", "}=",
                ";-{{", ";-{", ":-{{", ":-{",
                ":-(", ";-(",
                ":,)", ":'{",
                "[:", ";]"
                ]

def getTweetEmoticons(tweet):
    print(tweet)
    regexp = {"SPACES": r"\s+"}
    emoji = list()
    for tok in re.split(ParseTweet.regexp["SPACES"], tweet.strip()):
        if tok in Emoticons.POSITIVE:
            emoji.append(tok)
        if tok in Emoticons.NEGATIVE:
            emoji.append(tok)
    return emoji

# # all emojis in a text 

def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)
tweets_df['emojis'] = [extract_emojis(tweet) for tweet in tweets_df['text'].values[i] 
                       for i in range(len(tweets_df['text']))]
#tweets_df['emojis'] =[getTweetEmoticons(text) for text in (tweets_df['text'].values[i]) 
#                      for i in range(len(tweets_df['text']))]  



ValueError: Length of values does not match length of index

In [32]:
test_text = tweets_df.text[:5]
for i in test_text:
    print(i)

1000 liters of pure water costs 1
For those worried about running out of fresh water it may help to know that desalination only costs 01 cents per 
Ah yes fair point
Used to live in Silicon Valley now I live in Silicone Valley
Not yet


In [34]:
# clean tweets 
tweets_df['text'] = [re.sub(r'http[A-Za-z0-9:/.]+','',str(tweets_df['text'][i])) for i in range(len(tweets_df['text']))]
removeHTML_text = [BeautifulSoup(tweets_df.text[i], 'lxml').get_text() for i in range(len(tweets_df.text))]
tweets_df.text = removeHTML_text
tweets_df['text'] = [re.sub(r'@[A-Za-z0-9]+','',str(tweets_df['text'][i])) for i in range(len(tweets_df['text']))]


weird_characters_regex = re.compile(r"[^\w\d ]") 
tweets_df.text = tweets_df.text.str.replace(weird_characters_regex, "")


# look at results 
display(tweets_df.text[0])
display(tweets_df.head(20))

'1000 liters of pure water costs 1'

,text,num_hashtags,all_upper,emojis,word_count,char_count
0,1000 liters of pure water costs 1,0,0,,7,33
1,For those worried about running out of fresh w...,0,0,,22,113
2,Ah yes fair point,0,0,,4,17
3,Used to live in Silicon Valley now I live in S...,0,1,,12,60
4,Not yet,0,0,,2,7
5,trouble,0,0,,2,8
6,Earth is,0,0,,2,8
7,Earths crust is only 1 of Earth mass so techni...,0,0,,16,77
8,Earth should be called Water Our surface is 71...,0,0,,19,94
9,Yeah good idea to offer that as a setting,0,0,,9,41


[]

In [23]:
# get word coult, char count
tweets_df['word_count'] = tweets_df['text'].apply(lambda x: len(str(x).split(" ")))
tweets_df['char_count'] = tweets_df['text'].str.len() ## this also includes spaces
display(tweets_df[['text','char_count','word_count']].head())

# check data types 
display(tweets_df.dtypes)

,text,char_count,word_count
0,1000 liters of pure water costs 1,33,7
1,For those worried about running out of fresh w...,113,22
2,Ah yes fair point,17,4
3,Used to live in Silicon Valley now I live in S...,60,12
4,Not yet,7,2


text            object
num_hashtags     int64
all_upper        int64
emojis          object
word_count       int64
char_count       int64
dtype: object

In [ ]:
# get average word length
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

tweets_df['avg_word'] = tweets_df['text'].apply(lambda x: avg_word(x))
tweets_df[['text','avg_word']].head()

In [ ]:
# all lowercase 
tweets_df['text'] = tweets_df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
display(tweets_df['text'].head())

In [ ]:
# remove stopwords and punctuation
retweet = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + retweet
#stop = stopwords.words('english')
tweets_df['text'] = tweets_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stoplist))
tweets_df['text'].head()

In [ ]:
# add sentiment feature
tweets_df['sentiment'] = tweets_df['text'].apply(lambda x: TextBlob(x).sentiment[0])
tweets_df['polarity'] = tweets_df['text'].apply(lambda x: TextBlob(x).sentiment[1])
tweets_df = tweets_df.sort_values(['sentiment'])

In [ ]:
display(tweets_df[['text','sentiment','polarity']].head())

In [ ]:
# add list of nouns
tweets_df['nouns'] = [TextBlob(tweets_df.text[i]).noun_phrases for i in range(len(tweets_df.text))]

In [ ]:
tweets_df['classify'] = [TextBlob(tweets_df.text[i]).noun_phrases for i in range(len(tweets_df.text))]

In [ ]:
# look at 10 most frequent words
freq = pd.Series(' '.join(tweets_df['text']).split()).value_counts()[:10]
print(freq)

# look at 10 least frequent words
freq = pd.Series(' '.join(tweets_df['text']).split()).value_counts()[-20:]
freq

In [ ]:
# adjust spelling using TextBlob
# run once when data are ready- takes a long time! 
#tweets_df['text'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
#tokenization
TextBlob(tweets_df['text'][0]).words

In [ ]:
# convert word to base form
tweets_df['text'] = tweets_df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
# build n-grams. That is, build some structures that often come together. For example, for Elon Musk, "Tesla 3."
# or for Trump, "build the wall," etc. 

TextBlob(tweets_df['text'][0]).ngrams(2)

In [ ]:
# TF-IDF explained https://www.analyticsvidhya.com/blog/2015/04/information-retrieval-system-explained/

# sample term frequency table
tf1 = (tweets_df['text'][0:10]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1.sort_values(['tf', 'words'], ascending=[0, 1]).head(10)

In [ ]:
# investigate which words appear frequently across texts -- these words don't give us much information (we,great,how,etc.)
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(tweets_df.shape[0]/(len(tweets_df[tweets_df['text'].str.contains(word)])))
display(tf1.sort_values(['idf'],ascending=[1]).head(10))

In [ ]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
display(tf1.sort_values(['tfidf'],ascending=[1]).head(10))

# can also do this with sklearn, see code below
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
#  stop_words= 'english',ngram_range=(1,1))
# train_vect = tfidf.fit_transform(tweets_df['text'])

# train_vect

In [ ]:
# build corpus
corpus=[]
a=[]
for i in range(len(tweets_df['text'])):
        a=tweets_df['text'][i]
        corpus.append(a)
        
# look at first 5 lines        
corpus[0:5]

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# remove common words and tokenize
# uncomment and download stopwords if nessisary
# nltk.download()
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

# tokenize words 
texts = [[word for word in str(document).split()] for document in corpus]

In [ ]:
print(texts[1])

In [ ]:
# build dictionary of words, save it 
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'twitter.dict'))

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'twitter.mm'), corpus)  # store to disk, for lda

# TfidfModel: multiplies a local component (term frequency) with a global component 
# (inverse document frequency), and normalizing the resulting documents to unit length
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

total_topics = 5
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

#Show first n important word in the topics:
lda.show_topics(total_topics,5)

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}

#made dataframe
df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)



In [ ]:
%matplotlib inline

g=sns.clustermap(df_lda.corr(), center=0, cmap="RdBu", metric='cosine', linewidths=.75, figsize=(12, 12))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()

In [ ]:
# look at results
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

Possible features to explore:

-Topic

-Diversity of words

-Diversity of topics

-Word density (words/tweet)

-Top words

In [ ]:
# plot tweet activity

trace = go.Histogram(
    x=tweetsT,
    marker=dict(
        color='blue'
    ),
    opacity=0.75
)

layout = go.Layout(
    title='Tweet Activity Over Years',
    height=450,
    width=1200,
    xaxis=dict(
        title='Month and year'
    ),
    yaxis=dict(
        title='Tweet Quantity'
    ),
    bargap=0.2,
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)